<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/NAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a simple controller to generate architecture parameters
class Controller(nn.Module):
    def __init__(self, search_space_size):
        super(Controller, self).__init__()
        self.fc = nn.Linear(search_space_size, search_space_size)

    def forward(self, x):
        return torch.softmax(self.fc(x), dim=-1)

# Define a simple model for the generated architecture
class SimpleModel(nn.Module):
    def __init__(self, num_units):
        super(SimpleModel, self).__init__()
        layers = [nn.Linear(1, num_units[0]), nn.ReLU()]
        for i in range(1, len(num_units)):
            layers.append(nn.Linear(num_units[i-1], num_units[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(num_units[-1], 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Define the search space and initialize the controller
search_space_size = 5  # Number of possible unit sizes
controller = Controller(search_space_size)

# Define the reinforcement learning loop for NAS
num_epochs = 100
learning_rate = 0.01
optimizer = optim.Adam(controller.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    controller_input = torch.randn(1, search_space_size)
    architecture_params = controller(controller_input).squeeze(0)
    architecture = torch.argmax(architecture_params).item()

    # Sample architecture parameters and train the model
    num_units = [int(2 ** (architecture + 1)) for _ in range(search_space_size)]
    model = SimpleModel(num_units)
    optimizer_model = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()

    # Train the model on some synthetic data
    for _ in range(10):
        x = torch.rand((32, 1))
        y = 3 * x + 2 + 0.1 * torch.randn_like(x)
        optimizer_model.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer_model.step()

    # Compute the reward as the inverse of the loss
    reward = -loss.item()

    # Update the controller using the reward
    optimizer.zero_grad()
    loss_controller = -torch.sum(torch.log(architecture_params) * reward)
    loss_controller.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Reward: {reward:.4f}')